In [ ]:
'''
The notebook creates a mp3 audio file with a summary in Spanish of the news and
the weather for the given regions. It costs around 0,1€ using OpenAI's API.

It follows the guide at https://www.datacamp.com/code-along/working-with-apis-in-python
'''

In [94]:
from google.colab import userdata
API_KEY_NEWSAPI = userdata.get('API_KEY_NEWSAPI')
API_KEY_METEOSOURCE = userdata.get('API_KEY_METEOSOURCE')
API_KEY_OPENAI = userdata.get('API_KEY_OPENAI')

In [2]:
import requests

In [ ]:
COUNTRY_FOR_NEWS = "us" # Spain is not supported
CITY_FOR_WEATHER = "Barcelona"

# Get Newsletter info

In [77]:
# Prepare API request for Newsapi
news_api_url_parameters = {
    'apiKey': API_KEY_NEWSAPI,
    'country': COUNTRY_FOR_NEWS
}
# It doesn't have support for any spanish newspaper. Gnews doesn't either.

In [100]:
# Get headline articles for a given country from Newsapi
news_response = requests.get('https://newsapi.org/v2/top-headlines',
                             news_api_url_parameters)

if news_response.status_code == 200:
  news_response_data = news_response.json()

  headline_articles = [
      {'title': article['title'],
       'description': article['description']}
      for article in news_response_data['articles'][:10]
  ]

  print(headline_articles[0])
else:
  print(news_response.text)

{'title': 'A ‘worst-case’ scenario: How giant insurance losses from L.A. fires could affect all Californians - San Francisco Chronicle', 'description': "The enormous wildfires burning through Los Angeles County are likely to worsen California's already bad insurance crisis, just as reforms were taking effect."}


In [79]:
# Prepare API request for Meteosource to find id of a place
meteosource_api_headers = {
    'X-API-Key': API_KEY_METEOSOURCE
}

meteosource_findplaces_url_parameters = {
    'text': CITY_FOR_WEATHER
}

In [80]:
# Get id of the place where we want to know the forecast
meteosource_response = requests.get(
    'https://www.meteosource.com/api/v1/free/find_places',
    params = meteosource_findplaces_url_parameters,
    headers = meteosource_api_headers)

if meteosource_response.status_code == 200:
  meteosource_response_data = meteosource_response.json()

  place_id = meteosource_response_data[0]['place_id']
  print(place_id)

else:
  print(meteosource_response.text)

barcelona


In [82]:
# Prepare API request for Meteosource to get the weather
meteosource_point_url_parameters = {
    'place_id': place_id,
    'sections': 'daily',
}

In [83]:
# Get id of the place where we want to know the forecast
meteosource_response = requests.get(
    'https://www.meteosource.com/api/v1/free/point',
    params = meteosource_point_url_parameters,
    headers = meteosource_api_headers)

if meteosource_response.status_code == 200:
  meteosource_response_data = meteosource_response.json()

  weather_forecast = meteosource_response_data['daily']['data'][0]['summary']
  print(weather_forecast)

else:
  print(meteosource_response.text)

Partly sunny, fewer clouds in the afternoon. Temperature 5/18 °C.


# Write Chat GPT Prompt

In [127]:
# Write the ChatGPT prompt to write a daily update
developer_message = '''
You are an AI assistant that sends a morning newsletter with the main news of
the day and the weather. You provide an enjoyable an easy-to-read text to
accompany user on their morning coffee. The update should be about 2-5 minutes
long, incorporating both the weather forecast and the top 10 headlines in a way
that feels conversational, lively and fits a specific tone (such as funny,
serious, sarcastic or motivational). You have to speak in Castillian Spanish
as it would be spoken in the country of Spain. For example, they do not use the
third person when talking to strangers, so you should use the second person. You
have to speak in Spain and try to sound natural in that language, even if the
information that you receive is written in english. Do not output anything else
than the text. The text will be sent to a text-to-speech api to generate an MP3,
so make sure the output does not contain something that should not be read out
loud.

Structure the monologue as follows:

1. Greeting: Start with a warm and welcoming greeting.
2. Weather Summary: Describe the day's weather, infusing the chosen tone to
make it engaging.
3. News Headlines: Present each headline in the chosen tone, followed with a
summary of the headline to give the user a deeper insight into the headline.
4. Closing: Wrap up with a concluding remark that leaves the user with a smile,
positive thought or playful nudge.

Be creative in how you incorporate the tone and style, ensuring that the text
is engaging and enjoyable to listen to.
'''

user_message = f'''
Please generate a 'Morning update' text in a funny and light tone.

Here is the Weater Forecast:
{weather_forecast}

Here are the News Headlines in JSON format:
{headline_articles}

Generate the text as specified in the system prompt, in spanish, following the
structure of greeting, weather summary, news headlines and closing remark.
'''

In [128]:
# Set OpenAI api parameters for the request
openai_headers = {
    'Authorization': f'Bearer {API_KEY_OPENAI}'
}

openai_request_parameters = {
    "model": "gpt-4o-mini",
    "messages": [
        {"role": "developer", "content": developer_message},
        {"role": "user", "content": user_message}
    ]
}

In [129]:
# Get the morning newsletter from OpenAi API
openai_response = requests.post('https://api.openai.com/v1/chat/completions',
                         headers = openai_headers,
                         json = openai_request_parameters)

if openai_response.status_code == 200:
  openai_response_data = openai_response.json()

else:
  print(openai_response.text)

In [130]:
openai_response_text = openai_response_data['choices'][0]['message']['content']
print(openai_response_text)

¡Buenos días, querido oyente! Espero que estés listo para comenzar el día con una dosis de risas y noticias frescas, como un café bien cargado por la mañana. 

Primero, hablemos del clima. Hoy es uno de esos días en los que el sol se esfuerza por salir y las nubes deciden quedarse un ratito más en la cama. Tendremos temperaturas que van de los 5 a los 18 grados, así que asegúrate de vestirte en capas, para que no acabes con el sudor de un maratonista en pleno verano en pleno invierno. 

Ahora, vamos con las novedades del día, que están más interesantes que ese gato que se ha hecho viral en internet:

1. **¿Un escenario de pesadilla?** Los incendios en Los Ángeles están quemando más que solo árboles; también están avivando una crisis de seguros. Parece que el clima y la economía californiana decidieron hacer un cóctel explosivo, justo cuando empezaban a hacer reformas. ¡Menuda fiesta!

2. **Jean Smart lanza un grito al cielo:** La actriz ha pedido a las cadenas de televisión que no tran

In [131]:
# Convert text to speech using OpenAi AI
openai_audio_parameters = {
    "model": "tts-1",
    "voice": "alloy",
    "input": openai_response_text,
}

openai_response = requests.post('https://api.openai.com/v1/audio/speech',
                         headers = openai_headers,
                         json = openai_audio_parameters)

if openai_response.status_code == 200:
  speech_file = open("morning_update.mp3", "wb")
  speech_file.write(openai_response.content)
  speech_file.close()

else:
  print(openai_response.text)